In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os
import shutil

In [3]:
path = '../data/datasets/meetup/'
events = os.path.join(path,'meta-events.csv')
members = os.path.join(path,'meta-members.csv')
rsvps = os.path.join(path,'rsvps.csv')
df_e =  pd.read_csv(events)
df_m = pd.read_csv(members)
df_r = pd.read_csv(rsvps)
hyperedges = {}


In [37]:
event_names = {}
for f in df_e.iterrows():
    e_id = f[1][0]
    e_name = f[1][2]
    event_names[e_id] = e_name

In [33]:
member_names = {}
for f in df_m.iterrows():
    m_id = f[1][0]
    m_name = f[1][1]
    member_names[m_id] = m_name

In [23]:
attend_dict = {}
for f in df_r.iterrows():
    # print(f)
    ev_id = f[1][1]
    m_id = f[1][2]
    attend_dict[ev_id] = attend_dict.get(ev_id,[])+[m_id]
for f in df_m.iterrows():
    m_id = f[1][1]
    m_name = f[1][2]
    print(m_id, ' ',m_name)

In [41]:
count=0
for k,v in attend_dict.items():
    if len(v)>1 and len(v)<5:
        count+=1
        print('event: ', event_names[k],' participants: ',v,' names: ',[member_names[x] for x in v])
        print(count)

event:  Schedule Meetup  participants:  [199563568, 208888939]  names:  ['Hallie Wiedner', 'Leah']
1
event:  Tennessee Craft Fair on Sunday Afternoon 9/24  participants:  [1720391, 194158019, 186264694]  names:  ['Kathryn', 'Mark', 'Mary Ann L']
2
event:  Trivia Night at Plantation Pub  participants:  [1720391, 194158019]  names:  ['Kathryn', 'Mark']
3
event:  Happy Hour Wednesdays at Jonathan's in Bellevue  participants:  [1720391, 186264694]  names:  ['Kathryn', 'Mary Ann L']
4
event:  Happy Hour Wednesdays at Jonathan's in Bellevue Sept. 21  participants:  [1720391, 194158019, 186264694]  names:  ['Kathryn', 'Mark', 'Mary Ann L']
5
event:  Happy Hour Wednesdays at Jonathan's in Bellevue  participants:  [1720391, 194158019, 186264694]  names:  ['Kathryn', 'Mark', 'Mary Ann L']
6
event:  Franklin/Nashville Parental Alienation Support  participants:  [162878732, 228129103]  names:  ['Cynthia Stone', 'Guy Gutierrez']
7
event:  Franklin/Nashville Parental Alienation Support  participants

KeyError: 186309120